<a href="https://colab.research.google.com/github/VenkateshDas/legal_text_retrieval_coliee/blob/master/parse_statute_law.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os 
os.chdir('/content/drive/My Drive/COLIEE Retrieval /Data')

#Parsing civil code xml

In [0]:
file_path = '/content/drive/My Drive/COLIEE Retrieval /Data/statute_law.xml'

In [0]:
import xml.etree.ElementTree as ET

tree = ET.parse(file_path)
root = tree.getroot()


In [0]:
import re 
pattern = re.compile('\t*')
i = 0
article_num = []
article_id = []
string_list = []
for article in root.iter('Article'):
    article_num.append(article.attrib['Num'])
    article_id.append(article.attrib['id'])
    string_var = []
    string = ''
    for title in article:
        temp1 = title.text
        temp1 = re.sub(pattern,'',temp1)
        temp1 = re.sub('\n','',temp1)
        # print(temp1)
        # string_var.append(title.text)
        string_var.append(temp1)
        for paragraph in title.iter('Paragraph'):
            temp2 = paragraph.text
            temp2 = re.sub(pattern,'',temp2)
            
            # string_var.append(paragraph.text)
            # print(temp2) 
            string_var.append(temp2) 
            for sentence in paragraph.iter('Sentence'):
                temp3 = sentence.text
                temp3 = re.sub(pattern,'',temp3)
                # print(temp3)
                string_var.append(temp3)
    string = ''.join(string_var)
    string_list.append(string)


In [7]:
print(len(article_num))
print(len(article_id))
print(len(string_list))

782
782
782


In [8]:
import pandas as pd 
dict = {'Number' : article_num , 'ID' : article_id, 'Article' : string_list }
df = pd.DataFrame(dict)
for i in range(len(df['Article'])):
    df['Article'][i] = re.sub('\n',' ',df['Article'][i])
df.head()

,Number,ID,Article
0,1,en_pt1ch1at1,(Fundamental Principles)Article 1 Private righ...
1,2,en_pt1ch1at2,(Standards for Construction)Article 2 This Cod...
2,3,en_pt1ch2sc1at1,Article 3 The enjoyment of private rights comm...
3,3_2,en_pt1ch2sc2at1,Article 3-2 If the person making a juridical a...
4,4,en_pt1ch2sc3at1,(Age of Majority)Article 4 The age of majority...


In [0]:
df.to_csv('statute_law_parsed.csv')

In [0]:
import pickle

f = open('statute_law_parsed_df.pkl','wb')
pickle.dump(df,f)

In [11]:
files = open('/content/drive/My Drive/COLIEE Retrieval /Data/statute_law_parsed_df.pkl','rb')
df_pkl = pickle.load(files)
df_pkl.head()

,Number,ID,Article
0,1,en_pt1ch1at1,(Fundamental Principles)Article 1 Private righ...
1,2,en_pt1ch1at2,(Standards for Construction)Article 2 This Cod...
2,3,en_pt1ch2sc1at1,Article 3 The enjoyment of private rights comm...
3,3_2,en_pt1ch2sc2at1,Article 3-2 If the person making a juridical a...
4,4,en_pt1ch2sc3at1,(Age of Majority)Article 4 The age of majority...


# Parsing query and articles XML

In [0]:
relevant_doc_file_path = '/content/drive/My Drive/COLIEE Retrieval /COLIEE2019-StatuteLaw-English Training Data/'

In [18]:
pattern = re.compile('Article\s\d*\-?\d')

PairID_arr= []
text1_arr = []
text2_arr = []
label_arr = []
article_numbers = []

for files in os.listdir(relevant_doc_file_path):
    if files.endswith('.xml'):
        print(files)
        xmlp = ET.XMLParser(encoding="utf-8")
        tree = ET.parse(relevant_doc_file_path+files, parser = xmlp)
        root = tree.getroot()

        for pair in root.findall('pair'):
            i = 0
            j = 0
            PairID = pair.get('id')
            PairID_arr.append(PairID)
            
            text1 = pair.find('t1').text
            text1_arr.append(text1)

            numbers = re.findall(pattern,text1)
            article_numbers.append(numbers)
            
            text2 = pair.find('t2').text
            text2_arr.append(text2)

            Label = pair.get('label')
            label_arr.append(Label)

riteval_H26.xml
riteval_H21.xml
riteval_H18.xml
riteval_H23.xml
riteval_H27.xml
riteval_H28.xml
riteval_H29.xml
riteval_H22.xml
riteval_H24.xml
riteval_H19.xml
riteval_H20.xml
riteval_H25.xml


In [19]:
dict_relevant = {'ID' : PairID_arr , 'Query' : text2_arr , 'Article_numbers' : article_numbers , 'Articles' : text1_arr }
df_relevant = pd.DataFrame(dict_relevant)
df_relevant.head()

,ID,Query,Article_numbers,Articles
0,H26-1-A,"\nIf there is a guardian of a minor, the minor...","[Article 737, Article 838]",\nArticle 737 \n(1) A minor shall obtain the c...
1,H26-1-C,\nA will made by an adult ward may be rescinde...,"[Article 962, Article 5, Article 9, Article 13...","\nArticle 962 \nThe provisions of Article 5, A..."
2,H26-1-E,\nThe family court may order the commencement ...,"[Article 11, Article 7]",\nArticle 11 \nWith respect to any person who ...
3,H26-1-G,\nWith respect to any act which requires the c...,"[Article 13, Article 9, Article 2, Article 602...",\nArticle 13 \n(1) A person under curatorship ...
4,H26-1-I,\nThe ruling of commencement of assistance at ...,"[Article 15, Article 7, Article 11, Article 17...",\nArticle 15 \n(1) With respect to any person ...


In [23]:
type(df_relevant['Article_numbers'])

pandas.core.series.Series

In [26]:
article_pattern = re.compile('\d\d?\d?\d?\-?\d?')
for i in range(len(df_relevant['Article_numbers'])):
    for j in range(len(df_relevant['Article_numbers'][i])):
        df_relevant['Article_numbers'][i][j] = re.sub('Article\s','',df_relevant['Article_numbers'][i][j])
    df_relevant['Article_numbers'][i] = set(df_relevant['Article_numbers'][i])
df_relevant.head()

,ID,Query,Article_numbers,Articles
0,H26-1-A,"\nIf there is a guardian of a minor, the minor...","{838, 737}",\nArticle 737 \n(1) A minor shall obtain the c...
1,H26-1-C,\nA will made by an adult ward may be rescinde...,"{5, 9, 17, 962, 13}","\nArticle 962 \nThe provisions of Article 5, A..."
2,H26-1-E,\nThe family court may order the commencement ...,"{7, 11}",\nArticle 11 \nWith respect to any person who ...
3,H26-1-G,\nWith respect to any act which requires the c...,"{602, 9, 11, 2, 13}",\nArticle 13 \n(1) A person under curatorship ...
4,H26-1-I,\nThe ruling of commencement of assistance at ...,"{7, 876-9, 17, 11, 15}",\nArticle 15 \n(1) With respect to any person ...


In [25]:
df_relevant['Articles'][3]

'\nArticle 13 \n(1) A person under curatorship must obtain the consent of his/her curator if he/she intends to perform any of the following acts;provided, however, that, this shall not apply to the acts provided for in the proviso of Article 9:\n(i) receive or use any principal;\n(ii) borrow any money or guarantee any obligation;\n(iii) perform any act with the purpose of obtaining or relinquishing any right regarding real estate or other valuable property;\n(iv) take any procedural action;\n(v) make a gift, make any settlement, or agree to arbitrate (referring to the agreement to arbitrate as provided in paragraph 1, Article 2 of the Arbitration Act (Act No. 138 of 2003));\n(vi) accept or renounce any inheritance, or partition any estate;\n(vii) refuse an offer of a gift, renounce any bequest, accept the offer of gift with burden, or accept any bequest with burden;\n(viii) effect any new construction, renovation, expansion, or major repairs; or\n(ix) make any lease agreement with a te

In [0]:
df_relevant.to_csv('query_relevant_articles_parsed.csv')

In [0]:
f = open('query_relevant_articles_parsed_df.pkl','wb')
pickle.dump(df_relevant,f)

In [29]:
files_relevant = open('/content/drive/My Drive/COLIEE Retrieval /Data/query_relevant_articles_parsed_df.pkl','rb')
df_relevant_pkl = pickle.load(files_relevant)
df_relevant_pkl.head()

,ID,Query,Article_numbers,Articles
0,H26-1-A,"\nIf there is a guardian of a minor, the minor...","{838, 737}",\nArticle 737 \n(1) A minor shall obtain the c...
1,H26-1-C,\nA will made by an adult ward may be rescinde...,"{5, 9, 17, 962, 13}","\nArticle 962 \nThe provisions of Article 5, A..."
2,H26-1-E,\nThe family court may order the commencement ...,"{7, 11}",\nArticle 11 \nWith respect to any person who ...
3,H26-1-G,\nWith respect to any act which requires the c...,"{602, 9, 11, 2, 13}",\nArticle 13 \n(1) A person under curatorship ...
4,H26-1-I,\nThe ruling of commencement of assistance at ...,"{7, 876-9, 17, 11, 15}",\nArticle 15 \n(1) With respect to any person ...


In [30]:
df_relevant_pkl.shape

(716, 4)